## Grabbing the Data
-----

### Setup:
---

In [56]:
# Import dependencies
import requests
from bs4 import BeautifulSoup
import json
from jsonpath_ng.ext import parse
import pandas as pd

In [57]:
# Create funtion for grabbing json sections
def getJSONSection(json_data, section):
    return [obj for obj in json_data if obj['name'] == section]

In [58]:
# Create function for grabbing json values
def getJSONValue(json_data, section, stat):
    section_data = getJSONSection(json_data['splits']['categories'], section)
    stat_data = getJSONSection(section_data[0]['stats'], stat)
    return stat_data[0]['displayValue']

### Grab Data:
---

In [60]:
# converts team name retrieved from statistics API to the team name used by the league for looking up player photos
def getTeamUrl(team_name):
    # laliga
    if (team_name == 'Real Sociedad'):
        return 'real-sociedad'
    if (team_name == 'Villarreal'):
        return 'villarreal-cf'
    if (team_name == 'Valencia'):
        return 'valencia-cf'
    if (team_name == 'Almería'):
        return 'ud-almeria'
    if (team_name == 'Mallorca'):
        return 'rcd-mallorca'
    if (team_name == 'Sevilla'):
        return 'sevilla-fc'
    if (team_name == 'Athletic Club'):
        return 'athletic-club'
    if (team_name == 'Rayo Vallecano'):
        return 'rayo-vallecano'
    if (team_name == 'Cádiz'):
        return 'cadiz-cf'
    if (team_name == 'Atletico Madrid'):
        return 'atletico-de-madrid'
    if (team_name == 'Granada'):
        return 'granada-cf'
    if (team_name == 'Real Madrid'):
        return 'real-madrid'
    if (team_name == 'Real Betis'):
        return 'real-betis'
    if (team_name == 'Celta Vigo'):
        return 'rc-celta'
    if (team_name == 'Getafe'):
        return 'getafe-cf'
    if (team_name == 'Osasuna'):
        return 'c-a-osasuna'
    if (team_name == 'Girona'):
        return 'girona-fc'
    if (team_name == 'Las Palmas'):
        return 'ud-las-palmas'
    if (team_name == 'Barcelona'):
        return 'fc-barcelona'
    if (team_name == 'Alavés'):
        return 'd-alaves'

    # bundesliga
    if (team_name == 'TSG Hoffenheim'):
          return 'tsg-hoffenheim'
    if (team_name == '1. FC Union Berlin'):
        return '1-fc-union-berlin'
    if (team_name == 'Borussia Dortmund'):
        return 'borussia-dortmund'
    if (team_name == 'Bayern Munich'):
        return 'fc-bayern-muenchen'
    if (team_name == 'FC Cologne'):
        return '1-fc-koeln'
    if (team_name == 'VfB Stuttgart'):
        return 'vfb-stuttgart'
    if (team_name == 'Eintracht Frankfurt'):
        return 'eintracht-frankfurt'
    if (team_name == '1. FC Heidenheim 1846'):
        return '1-fc-heidenheim-1846'
    if (team_name == 'RB Leipzig'):
        return 'rb-leipzig'
    if (team_name == 'Borussia Monchengladbach'):
        return 'borussia-moenchengladbach'
    if (team_name == 'SC Freiburg'):
        return 'sport-club-freiburg'
    if (team_name == 'Mainz'):
        return '1-fsv-mainz-05'
    if (team_name == 'Bayer Leverkusen'):
        return 'bayer-04-leverkusen'
    if (team_name == 'Werder Bremen'):
        return 'sv-werder-bremen'
    if (team_name == 'VfL Bochum'):
        return 'vfl-bochum-1848'
    if (team_name == 'VfL Wolfsburg'):
        return 'vfl-wolfsburg'
    if (team_name == 'FC Augsburg'):
        return 'fc-augsburg'
    if (team_name == 'SV Darmstadt 98'):
        return 'sv-darmstadt-98'

    # liga mx
    if (team_name == 'Pumas UNAM'):
        return "18/pumas"
    if (team_name == 'Monterrey'):
         return "14/monterrey"
    if (team_name == 'FC Juarez'):
         return "11790/fc-juarez"
    if (team_name == 'Pachuca'):
         return "11/pachuca"
    if (team_name == 'Querétaro'):
         return "12037/queretaro"
    if (team_name == 'Mazatlán FC'):
         return "12043/mazatlan-fc"
    if (team_name == 'Tigres UANL'):
        return '16/tigres'
    if (team_name == 'León'):
        return '9/leon'
    if (team_name == 'América'):
        return '1/america'
    if (team_name == 'Cruz Azul'):
        return '12566/cruz-azul'
    if (team_name == 'Atlético de San Luis'):
        return '11220/atletico-de-san-luis'
    if (team_name == 'Tijuana'):
        return '5/tijuana'
    if (team_name == 'Puebla'):
        return '11550/puebla'
    if (team_name == 'Guadalajara'):
        return '7/guadalajara'
    if (team_name == 'Toluca'):
        return '17/toluca'
    if (team_name == 'Atlas'):
        return '10445/atlas'
    if (team_name == 'Santos'):
        return '15/santos-laguna'
    if (team_name == 'Necaxa'):
        return '29/necaxa'
    

    # ligue 1
    if (team_name == 'Metz'):
          return "fc-metz"
    if (team_name == 'Nice'):
          return "ogc-nice"
    if (team_name == 'Paris Saint-Germain'):
         return "paris-saint-germain"
    if (team_name == 'Lille'):
        return "losc-lille"
    if (team_name == 'Lyon'):
          return "olympique-lyonnais"
    if (team_name == 'Stade Rennais'):
         return "stade-rennais-fc"
    if (team_name == 'Nantes'):
          return "fc-nantes"
    if (team_name == 'Montpellier'):
         return "montpellier-herault-sc"
    if (team_name == 'Strasbourg'):
        return 'rc-strasbourg-alsace'
    if (team_name == 'Clermont Foot'):
        return 'clermont-foot-63'
    if (team_name == 'Lens'):
        return 'rc-lens'
    if (team_name == 'Marseille'):
        return 'olympique-de-marseille'
    if (team_name == 'Le Havre AC'):
        return 'havre-ac'
    if (team_name == 'Brest'):
        return 'stade-brestois-29'
    if (team_name == 'Lorient'):
        return 'fc-lorient'
    if (team_name == 'AS Monaco'):
        return 'as-monaco'
    if (team_name == 'Stade de Reims'):
        return 'stade-de-reims'
    if (team_name == 'Toulouse'):
        return 'toulouse-fc'
    

    # serie A
    if (team_name == 'Hellas Verona'):
         return "VERONA"
    if (team_name == 'Genoa'):
         return "GENOA"
    if (team_name == 'Sassuolo'):
         return "SASSU"
    if (team_name == 'Bologna'):
         return "BOLO"
    if (team_name == 'Fiorentina'):
         return "FIORE"
    if (team_name == 'AS Roma'):
         return "ROMA"
    if (team_name == 'AC Milan'):
         return "MILAN"
    if (team_name == 'Atalanta'):
         return "ATALAN"
    if (team_name == 'Cagliari'):
         return "CAGLIA"
    if (team_name == 'Monza'):
        return 'MONZA'
    if (team_name == 'Internazionale'):
        return 'INTER'
    if (team_name == 'Udinese'):
        return 'UDINES'
    if (team_name == 'Salernitana'):
        return 'SALER'
    if (team_name == 'Lazio'):
        return 'LAZIO'
    if (team_name == 'Juventus'):
        return 'JUVE'
    if (team_name == 'Frosinone'):
        return 'FROSI'
    if (team_name == 'Empoli'):
        return 'EMPOLI'
    if (team_name == 'Torino'):
        return 'TORINO'
    if (team_name == 'Lecce'):
        return 'LECCE'
    if (team_name == 'Napoli'):
        return 'NAPOLI'
    

In [61]:
soccer_leagues = [
    {
        'display': 'MLS',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'MLS',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'NWSL',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/usa.nwsl/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Premier League',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/eng.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'LALIGA',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'LALIGA',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'Bundesliga',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/ger.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Bundesliga',
        'url': 'http://sports.core.api.espn.com/v2/sports/soccer/leagues/ger.1/seasons/2023/athletes?lang=en&region=us&limit=1000&page=2'
    },
    {
        'display': 'Liga MX',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/mex.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Ligue 1',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/fra.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    },
    {
        'display': 'Serie A',
        'url': 'https://sports.core.api.espn.com/v2/sports/soccer/leagues/ita.1/seasons/2023/athletes?lang=en&region=us&limit=1000'
    }
]

In [62]:
selected_leagues = ['LALIGA', 'Bundesliga', 'Liga MX', 'Ligue 1', 'Serie A']

In [66]:
#Setup for grabbing data

#Setup lists for stats
names = []
teams = []
leagues = []
positions = []
jersey_numbers = []
goals = []
assists = []
yellow_cards = []
red_cards = []
shots = []
shots_og = []
saves = []
clean_sheets = []
goals_against = []
photo_urls = []
heights = []
weights = []
birthdates = []
nationalities = []

for league in soccer_leagues:
   if (league['display'] in selected_leagues):
     athlete_list_results_json = requests.get(league['url']).json()
    
     #Loop through players
     for athlete in range(len(athlete_list_results_json['items'])):

       athlete_results = requests.get(athlete_list_results_json['items'][athlete]['$ref'])
       athlete_results_json = athlete_results.json()
       if ('statistics' in athlete_results_json and 'team' in athlete_results_json):  
         print(athlete_list_results_json['items'][athlete]['$ref'])
         team_results = requests.get(athlete_results_json['team']['$ref'])
         team_results_json = team_results.json()

         athlete_stats_results = requests.get(athlete_results_json['statistics']['$ref'])
         athlete_stats_results_json = athlete_stats_results.json()
        
         #Grab images:
         #premier league
         if league['display'] == 'Premier League':
           try:
             name_search_url = f"https://footballapi.pulselive.com/search/PremierLeague/?terms={athlete_results_json['displayName']}&type=player&size=1&start=0&fullObjectResponse=true"
             name_search_result = requests.get(name_search_url, headers = {"origin": "https://www.premierleague.com"})
             name_search_result_json = name_search_result.json()

             image_id = name_search_result_json['hits']['hit'][0]['response']['altIds']['opta']
             image_url = f"https://resources.premierleague.com/premierleague/photos/players/250x250/{image_id}.png"
           except:
             image_url = 'N/A'

         #laliga
         elif league['display'] == 'LALIGA':
           try:
             club_squad_url = f"https://www.laliga.com/en-US/clubs/{getTeamUrl(team_results_json['displayName'])}/squad"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
             club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': '__NEXT_DATA__'}).text)
             player_element = [obj for obj in club_squad_json['props']['pageProps']['squad']['squads'] if ('shirt_number' in obj) and (str(obj['shirt_number'])) == athlete_results_json['jersey']]
             image_url =  player_element[0]['photos']['002']['512x512']
           except:
             image_url = 'N/A'

         # bundesliga
         elif league['display'] == 'Bundesliga':
           try:
             club_squad_url = f"https://www.bundesliga.com/en/bundesliga/clubs/{getTeamUrl(team_results_json['displayName'])}"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))
             club_squad_json = json.loads(club_squad_results_parsed.body.find('script', attrs={'id': 'serverApp-state'}).text)
             jsonpath_expr = parse(f"$..players.*[?(@.shirtNumber={athlete_results_json['jersey']})]")
             match = [items.value['playerImages']['FACE_CIRCLE'].replace('-circle', '') for items in jsonpath_expr.find(club_squad_json)]
             image_url = match[0]
           except:
             image_url = 'N/A'

         # mls
         elif league['display'] == 'MLS':
           try:
             image_url = athlete_results_json['headshot']['href'] if 'headshot' in athlete_results_json else 'N/A'
           except:
             image_url = 'N/A'

         # nwsl
         elif league['display'] == 'NWSL':
           try:
             base_url = 'https://d2nkt8hgeld8zj.cloudfront.net'
             player_list_url = f'{base_url}/services/nwsl.ashx/players'

             player_list_results = requests.get(player_list_url)
             player_list_results_results_json = player_list_results.json()
                    
             jsonpath_expr = parse(f"$.data[?(@.shirtNumber=='{athlete_results_json['jersey']}' & @.team.title == '{team_results_json['displayName']}')]")
             match = [items.value['images']['head_shot']['url'] for items in jsonpath_expr.find(player_list_results_results_json)]
             image_url = f'{base_url}{match[0]}' if len(match) > 0 else 'N/A'
           except:
             image_url = 'N/A'

         # liga mx
         elif league['display'] == 'Liga MX':
           try:
             club_squad_url = f"https://ligamx.net/cancha/club/{getTeamUrl(team_results_json['displayName'])}"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))

             match = [number.parent.parent.parent.find('img') for number in club_squad_results_parsed.find_all('strong', {'class': 'numero'}) if number.get_text() == f"#{athlete_results_json['jersey']}"]
             image_url = match[0]['src'] if len(match) > 0 else 'N/A'
           except:
             image_url = 'N/A'

         # ligue 1
         elif league['display'] == 'Ligue 1':
           try:
             base_url = "https://www.ligue1.com"
             club_squad_url = f"{base_url}/clubs/squad?id={getTeamUrl(team_results_json['displayName'])}"
             club_squad_results = requests.get(club_squad_url)
             club_squad_results_parsed = BeautifulSoup(club_squad_results.content.decode('utf-8'))

             match = [number.parent.find('img') for number in club_squad_results_parsed.find_all('div', {'class': 'SquadTeamTable-detail--number'}) if number.get_text(strip=True) == athlete_results_json['jersey']]
             image_url = f"{base_url}{match[0]['src']}" if len(match) > 0 else 'N/A'
           except:
             image_url = 'N/A'

         # serie A
         elif league['display'] == 'Serie A':
           try:
             player_results = requests.get(f"https://www.legaseriea.it/api/team/{getTeamUrl(team_results_json['displayName'])}/players")
             player_results_json = player_results.json()

             jsonpath_expr = parse(f"$..data.*[?(@.uniform_number={athlete_results_json['jersey']})]")
             match = [items.value['medium_shot'] for items in jsonpath_expr.find(player_results_json)]
             image_url = match[0]
           except:
             image_url = 'N/A'
            
         # Grab stats for current player
         cur_name = athlete_results_json['displayName']
         cur_team = team_results_json['displayName']
         cur_league = league['display']
         cur_position = athlete_results_json['position']['displayName']
         cur_jersey_number = athlete_results_json['jersey']if 'jersey' in athlete_results_json else 'N/A'
         cur_goals = getJSONValue(athlete_stats_results_json, 'offensive', 'totalGoals')
         cur_assists = getJSONValue(athlete_stats_results_json, 'offensive', 'goalAssists')
         cur_yellow_cards = getJSONValue(athlete_stats_results_json, 'general', 'yellowCards')
         cur_red_cards = getJSONValue(athlete_stats_results_json, 'general', 'redCards')
         cur_shots = getJSONValue(athlete_stats_results_json, 'offensive', 'totalShots')
         cur_shots_og = getJSONValue(athlete_stats_results_json, 'offensive', 'shotsOnTarget')
         cur_saves = getJSONValue(athlete_stats_results_json, 'goalKeeping', 'saves')
         cur_clean_sheets = getJSONValue(athlete_stats_results_json, 'goalKeeping', 'cleanSheet')
         cur_goals_against = getJSONValue(athlete_stats_results_json, 'goalKeeping', 'goalsConceded')
         cur_photo_url = image_url
         cur_height = athlete_results_json['displayHeight'] if 'displayHeight' in athlete_results_json else 'N/A'
         cur_weight = athlete_results_json['displayWeight'] if 'displayWeight' in athlete_results_json else 'N/A'
         cur_birthdate = athlete_results_json['dateOfBirth']if 'dateOfBirth' in athlete_results_json else 'N/A'
         cur_nationality = athlete_results_json['citizenship']if 'citizenship' in athlete_results_json else 'N/A'
            
         # Print summary
         print('**********')
         print(f"Name: {cur_name}")
         print(f"Team: {cur_team}")
         print(f"League: {cur_league}")
         print(f"Position: {cur_position}")
         print(f"Jersey Number: {cur_jersey_number}")
         print(f"Goals: {cur_goals}")
         print(f"Assists: {cur_assists}")
         print(f"Yellow Cards: {cur_yellow_cards}")
         print(f"Red Cards: {cur_red_cards}")
         print(f"Shots: {cur_shots}")
         print(f"Shots on Goal: {cur_shots_og}")
         print(f"Saves: {cur_saves}")
         print(f"Clean Sheets: {cur_clean_sheets}")
         print(f"Goals Against: {cur_goals_against}")
         print(f"Image URL: {cur_photo_url}")
         print(f"Height: {cur_height}")
         print(f"Weight: {cur_weight}")
         print(f"Birthdate: {cur_birthdate}")
         print(f"Nationality: {cur_nationality}")

         # Append those stats to their respective lists
         names.append(cur_name)
         teams.append(cur_team)
         leagues.append(cur_league)
         positions.append(cur_position)
         jersey_numbers.append(cur_jersey_number)
         goals.append(cur_goals)
         assists.append(cur_assists)
         yellow_cards.append(cur_yellow_cards)
         red_cards.append(cur_red_cards)
         shots.append(cur_shots)
         shots_og.append(cur_shots_og)
         saves.append(cur_saves)
         clean_sheets.append(cur_clean_sheets)
         goals_against.append(cur_goals_against)
         photo_urls.append(cur_photo_url)
         heights.append(cur_height)
         weights.append(cur_weight)
         birthdates.append(cur_birthdate)
         nationalities.append(cur_nationality) 


http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes/3368?lang=en&region=us
**********
Name: André Silva
Team: Real Sociedad
League: LALIGA
Position: Forward
Jersey Number: 21
Goals: 0
Assists: 0
Yellow Cards: 0
Red Cards: 0
Shots: 2
Shots on Goal: 1
Saves: 0
Clean Sheets: 0
Goals Against: 0
Image URL: https://assets.laliga.com/squad/2023/t188/p165811/512x512/p165811_t188_2023_1_002_000.jpg
Height: 6' 1"
Weight: 183 lbs
Birthdate: 1995-11-06T08:00Z
Nationality: Portugal
http://sports.core.api.espn.com/v2/sports/soccer/leagues/esp.1/seasons/2023/athletes/12910?lang=en&region=us
**********
Name: Pepe Reina
Team: Villarreal
League: LALIGA
Position: Goalkeeper
Jersey Number: 1
Goals: 0
Assists: 0
Yellow Cards: 0
Red Cards: 0
Shots: 0
Shots on Goal: 0
Saves: 0
Clean Sheets: 0
Goals Against: 0
Image URL: https://assets.laliga.com/squad/2023/t449/p8432/512x512/p8432_t449_2023_1_002_000.jpg
Height: 6' 2"
Weight: 203 lbs
Birthdate: 1982-08-31T07:00Z
Nationality: 

### Check Success and Export:
---


In [65]:
# Check pulled data length
print (len(names))

545


In [47]:
# Create athlete_statistics dictionary with lists of data
athlete_statistics = {
    "player_name" : names,
    "team" : teams,
    "league" : leagues,
    "position" : positions,
    "jersey_number" : jersey_numbers,
    "goals" : goals,
    "assists" : assists,
    "yellow_cards" : yellow_cards,
    "red_cards" : red_cards,
    "shots" : shots,
    "shots_on_goal" : shots_og,
    "saves" : saves,
    "clean_sheets" : clean_sheets,
    "goals_against" : goals_against,
    "photo_url" : photo_urls,
    "height" : heights,
    "weight" : weights,
    "birthdate" : birthdates,
    "nationality" : nationalities
}

In [48]:
# View uncleaned dataframe
athelete_stats_df = pd.DataFrame(athlete_statistics)
athelete_stats_df

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height,weight,birthdate,nationality
0,André Silva,Real Sociedad,LALIGA,Forward,21,0,0,0,0,2,1,0,0,0,N/A,"6' 1""",183 lbs,1995-11-06T08:00Z,Portugal
1,Pepe Reina,Villarreal,LALIGA,Goalkeeper,1,0,0,0,0,0,0,0,0,0,N/A,"6' 2""",203 lbs,1982-08-31T07:00Z,Spain
2,Javi Guerra,Valencia,LALIGA,Midfielder,8,3,1,2,0,18,8,2,4,16,N/A,"6' 2""",170 lbs,2003-05-13T07:00Z,Spain
3,Alejandro Pozo,Almería,LALIGA,Defender,17,0,0,0,0,1,1,0,0,16,N/A,"5' 9""",152 lbs,1999-02-22T08:00Z,Spain
4,Iván Cuéllar,Mallorca,LALIGA,Goalkeeper,25,0,0,0,0,0,0,0,0,0,N/A,"6' 2""",168 lbs,1984-05-27T07:00Z,Spain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,João Gabriel E Costa Cesco,AS Roma,Serie A,Forward,67,0,0,0,0,0,0,0,0,0,N/A,"5' 10""",N/A,2005-03-28T08:00Z,Portugal
2623,Federico Mosca,Udinese,Serie A,Goalkeeper,70,0,0,0,0,0,0,0,0,0,N/A,"6' 4""",N/A,2005-08-08T07:00Z,Italy
2624,Nicolò Calabrese,Hellas Verona,Serie A,Defender,75,0,0,0,0,0,0,0,0,0,N/A,N/A,N/A,2004-11-16T08:00Z,Italy
2625,Simone Calvani,Genoa,Serie A,Goalkeeper,38,0,0,0,0,0,0,0,0,0,N/A,N/A,N/A,2005-05-16T07:00Z,Italy


In [42]:
# Export unclean data to CSV
athelete_stats_df.to_csv ("initial_unclean_csv/US_unclean_data.csv", encoding='utf8', index=False, header=True)
